In [ ]:
import h5py
import labrotation.file_handling as fh
import numpy as np
import os

In [ ]:
assembled_traces_fpath = fh.open_file("Open assembled_traces.h5 file!")

## Gather datasets

In [ ]:
dsets_dict = dict()
uuids_list = []
mouse_ids_list = []
win_types_list = []
nik_lengths_list = []
lfp_lengths_list = []
with h5py.File(assembled_traces_fpath, "r") as hf:
    for uuid in hf.keys():
        uuids_list.append(uuid)
        uuid_grp = hf[uuid]
        win_types_list.append(uuid_grp.attrs["window_type"])
        nik_lengths_list.append(uuid_grp.attrs["n_frames"])
        lfp_lengths_list.append(uuid_grp.attrs["n_lfp_steps"])
        mouse_ids_list.append(uuid_grp.attrs["mouse_id"])
        for dset_name in uuid_grp.keys():
            if dset_name not in dsets_dict.keys():
                dsets_dict[dset_name] = []
            dsets_dict[dset_name].append(uuid_grp[dset_name][:])

## Create 2d array with 0 padding at the end

In [ ]:
dsets_2d_dict = dict()
for dset_name in dsets_dict.keys():
    max_length = max([len(dset) for dset in dsets_dict[dset_name]])
    padded_dsets = [np.pad(dset, (0, max_length - len(dset)), 'constant') for dset in dsets_dict[dset_name]]
    dsets_2d_dict[dset_name] = np.vstack(padded_dsets)

## Save reformatted dataset

In [ ]:
output_fname = os.path.splitext(os.path.split(assembled_traces_fpath)[-1])[0] + "_matlab.h5"
output_fpath = os.path.join(os.path.split(assembled_traces_fpath)[0], output_fname)
print(f"Saving results to {output_fpath}")

In [ ]:
with h5py.File(output_fpath, "w") as hf:
    hf.create_dataset("uuids", data=uuids_list)
    hf.create_dataset("mouse_ids", data=mouse_ids_list)
    hf.create_dataset("win_types", data=win_types_list)
    hf.create_dataset("nik_lengths", data=nik_lengths_list)
    hf.create_dataset("lfp_lengths", data=lfp_lengths_list)
    for dset_name in dsets_2d_dict.keys():
        hf.create_dataset(dset_name, data=dsets_2d_dict[dset_name])